In [ ]:
def receiveMessage(self, currentTime, msg):
    super().receiveMessage(currentTime, msg)
    if self.state == 'AWAITING_SPREAD' and msg.body['msg'] == 'QUERY_SPREAD':
        dt = (self.mkt_close - currentTime) / np.timedelta64(1, 'm')
        if dt < 25:
            self.dump_shares()
        else:
            bid, _, ask, _ = self.getKnownBidAsk(self.symbol)
            if bid and ask:
                mid_price = (bid + ask) / 2
                self.mid_list.append(mid_price)
                if len(self.mid_list) > self.window1: 
                    self.avg_win1_list.append(pd.Series(self.mid_list).ewm(span=self.window1).mean().values[-1].round(2))
                if len(self.mid_list) > self.window2: 
                    self.avg_win2_list.append(pd.Series(self.mid_list).ewm(span=self.window2).mean().values[-1].round(2))

                # Updating bayesian estimation with mid_price as a new observation
                self.bayesian_update(mid_price)

                if len(self.avg_win1_list) > 0 and len(self.avg_win2_list) > 0 and len(self.orders) == 0:
                    # Get the bayesian signal
                    bayesian_signal = self.get_bayesian_signal()

                    # Trading strategy
                    if bayesian_signal == 'BUY' and self.avg_win1_list[-1] >= self.avg_win2_list[-1]:
                        # Check that we have enough cash to place the order
                        if self.holdings['CASH'] >= (self.size * ask):
                            self.placeLimitOrder(self.symbol, quantity=self.size, is_buy_order=True, limit_price=ask)
                    elif bayesian_signal == 'SELL' and self.avg_win1_list[-1] < self.avg_win2_list[-1]:
                        if self.symbol in self.holdings and self.holdings[self.symbol] > 0:
                            order_size = min(self.size, self.holdings[self.symbol])
                            self.placeLimitOrder(self.symbol, quantity=order_size, is_buy_order=False, limit_price=bid)
        self.setWakeup(currentTime + self.getWakeFrequency())
        self.state = 'AWAITING_WAKEUP'


In [ ]:
from agent.TradingAgent import TradingAgent
import pandas as pd
import numpy as np
import os
from contributed_traders.util import get_file

class ydeng335_DevonRex(TradingAgent):
    """
    Simple Trading Agent that compares the past mid-price observations and places a
    buy limit order if the first window mid-price exponential average >= the second window mid-price exponential average or a
    sell limit order if the first window mid-price exponential average < the second window mid-price exponential average
    """

    def __init__(self, id, name, type, symbol, starting_cash,
                min_size, max_size, wake_up_freq='60s',
                log_orders=False, random_state=None, mean_valuation=100, std_valuation=10):

        super().__init__(id, name, type, starting_cash=starting_cash, log_orders=log_orders, random_state=random_state)
        self.symbol = symbol
        self.min_size = min_size  # Minimum order size
        self.max_size = max_size  # Maximum order size
        self.size = self.random_state.randint(self.min_size, self.max_size)
        self.wake_up_freq = wake_up_freq
        self.mid_list, self.avg_win1_list, self.avg_win2_list = [], [], []
        self.log_orders = log_orders
        self.state = "AWAITING_WAKEUP"

        # Initialize histories
        self.order_history = []
        self.spread_history = []
        self.liquidity_history = []
        self.trade_executed=0
        # Initialize Bayesian parameters
        self.mean_valuation = mean_valuation
        self.std_valuation = std_valuation

        # Initialize Bayesian posterior parameters
        self.posterior_mean = mean_valuation
        self.posterior_std = std_valuation


    def kernelStarting(self, startTime):
        super().kernelStarting(startTime)
        # Read in the configuration through util
        with open(get_file('ydeng335_DevonRex/DevonRex.cfg'), 'r') as f:
            self.window1, self.window2 = [int(w) for w in f.readline().split()]
        #print(f"{self.window1} {self.window2}")


    # We use Bayesian updating in our last_trade method
    def last_trade(self, price, quantity, time):
        self.order_history.append(price)
        self.bayesian_update()
    
    # Similarly, we update our beliefs when we receive new spread and liquidity information
    def process_spread(self, bid, bid_vol, ask, ask_vol, time):
        spread = ask - bid
        self.spread_history.append(spread)
        self.bayesian_update()
    
    def process_liquidity(self, bid_liquidity, ask_liquidity, time):
        liquidity = bid_liquidity + ask_liquidity
        self.liquidity_history.append(liquidity)
        self.bayesian_update()

    def bayesian_update(self):
        # We need at least one data point to perform an update
        if len(self.order_history) == 0 or len(self.spread_history) == 0 or len(self.liquidity_history) == 0:
            return

        # Here we simply average our factors, but a more sophisticated agent could use different weights or a more complex function
        likelihood_mean = np.mean(self.order_history)
        spread_factor = np.mean(self.spread_history)
        liquidity_factor = np.mean(self.liquidity_history)

        # We adjust our standard deviation based on spread and liquidity
        # The more spread and less liquidity, the more uncertainty in our valuation
        adjusted_std = self.std_valuation * (1 + spread_factor / liquidity_factor)

        # Now we perform a standard Bayesian update
        self.posterior_mean = (likelihood_mean / adjusted_std**2 + self.mean_valuation / self.std_valuation**2) / (1 / adjusted_std**2 + 1 / self.std_valuation**2)
        self.posterior_std = np.sqrt(1 / (1 / adjusted_std**2 + 1 / self.std_valuation**2))

        # Update our beliefs (the "actual" mean and std dev)
        self.mean_valuation = likelihood_mean
        self.std_valuation = adjusted_std

    def get_bayesian_signal(self):
        """Generate trading signals based on the difference between the Bayesian posterior estimates and the actual values"""
        mean_diff = self.posterior_mean - self.mean_valuation
        std_diff = self.posterior_std - self.std_valuation
        
        if mean_diff > 0 and std_diff < 0:
            # If the Bayesian estimate of the mean is higher and less uncertain than the actual, we issue a 'BUY' signal
            return 'BUY'
        elif mean_diff < 0 and std_diff > 0:
            # If the Bayesian estimate of the mean is lower and more uncertain than the actual, we issue a 'SELL' signal
            return 'SELL'
        else:
            # In all other cases, we issue a 'HOLD' signal
            return 'HOLD'
        
    def wakeup(self, currentTime):
        """ Agent wakeup is determined by self.wake_up_freq """
        can_trade = super().wakeup(currentTime)
        if not can_trade: return
        self.state = 'AWAITING_SPREAD'
        spread = self.getCurrentSpread(self.symbol)
        if spread is None:
            # Handle case where spread is None
            # (for example, by returning, or by using a default value)
            return
        bid, bid_vol, ask, ask_vol = spread
        self.getCurrentSpread(self.symbol)

        # Get last trade using parent method
        last_trade_price = self.getLastTrade(self.symbol)
        
        # Get recent order stream using parent method
        recent_orders = self.getOrderStream(self.symbol)
        
        # Process data
        self.process_last_trade(last_trade_price)
        self.process_spread(bid, bid_vol, ask, ask_vol)
        self.process_orders(recent_orders)

    def dump_shares(self):
        # get rid of any outstanding shares we have
        if self.symbol in self.holdings and len(self.orders) == 0:
            order_size = self.holdings[self.symbol]
            bid, _, ask, _ = self.getKnownBidAsk(self.symbol)
            if bid:
                self.placeLimitOrder(self.symbol, quantity=order_size, is_buy_order=False, limit_price=0)

    def receiveMessage(self, currentTime, msg):
        """ Momentum agent actions are determined after obtaining the best bid and ask in the LOB """
        super().receiveMessage(currentTime, msg)
        if self.state == 'AWAITING_SPREAD' and msg.body['msg'] == 'QUERY_SPREAD':
            dt = (self.mkt_close - currentTime) / np.timedelta64(1, 'm')
            if dt < 25:
                self.dump_shares()
            else:
                bid, _, ask, _ = self.getKnownBidAsk(self.symbol)
                if bid and ask:
                    self.mid_list.append((bid + ask) / 2)
                    if len(self.mid_list) > self.window1: self.avg_win1_list.append(pd.Series(self.mid_list).ewm(span=self.window1).mean().values[-1].round(2))
                    if len(self.mid_list) > self.window2: self.avg_win2_list.append(pd.Series(self.mid_list).ewm(span=self.window2).mean().values[-1].round(2))
                    bayesian_signal = self.get_bayesian_signal()
                    if len(self.avg_win1_list) > 0 and len(self.avg_win2_list) > 0 and len(self.orders) == 0:
                        if self.avg_win1_list[-1] >= self.avg_win2_list[-1] and bayesian_signal == 'BUY':
                            # Check that we have enough cash to place the order
                            if self.holdings['CASH'] >= (self.size * ask):
                                self.placeLimitOrder(self.symbol, quantity=self.size, is_buy_order=True, limit_price=ask)
                        elif self.symbol in self.holdings and self.holdings[self.symbol] > 0 and bayesian_signal == 'SELL':
                            order_size = min(self.size, self.holdings[self.symbol])
                            self.placeLimitOrder(self.symbol, quantity=order_size, is_buy_order=False, limit_price=bid)
                        elif bayesian_signal == 'BUY':
                            # Check that we have enough cash to place the order
                            min_order_size = min(self.holdings['CASH'] // ask, self.min_size)
                            if min_order_size >= self.min_size:
                                self.placeLimitOrder(self.symbol, quantity=min_order_size, is_buy_order=True, limit_price=ask)
                        elif self.symbol in self.holdings and self.holdings[self.symbol] > 0 and bayesian_signal == 'SELL':
                            min_order_size = min(self.holdings[self.symbol], self.min_size)
                            if min_order_size >= self.min_size:
                                self.placeLimitOrder(self.symbol, quantity=min_order_size, is_buy_order=False, limit_price=bid)

            self.setWakeup(currentTime + self.getWakeFrequency())
            self.state = 'AWAITING_WAKEUP'

    def orderExecuted(self, order):
        super().orderExecuted(order)  # Call the parent class's method
        self.trade_executed += 1  # Then increment your new attribute


    def getWakeFrequency(self):
        return pd.Timedelta(self.wake_up_freq)

    def author():
        return 'ydeng335'
    
    def agentname():
        return 'ydeng335_DevonRex'
    
    def number_of_counting(self):
        return self.trade_executed

In [ ]:
    def get_bayesian_signal(self, currentTime):
        """Generate trading signals based on the Bayesian posterior estimates and the current price"""
        self.bayesian_update()  # Update beliefs before generating signal

        # Get Bollinger Band Percent
        bbp = self.get_bbp()

        # Get time to market close
        dt = (self.mkt_close - currentTime) / np.timedelta64(1, 'm')

        # Convert Timedelta to float (number of minutes)
        market_duration = (self.mkt_close - self.mkt_open).total_seconds() / 60

        # Compute decay factor based on time to market close
        decay_factor = dt / market_duration

        # Get current mid price and quantities
        bid, bid_qty, ask, ask_qty = self.getKnownBidAsk(self.symbol)
        mid_price = (bid + ask) / 2 if bid and ask else 0
        current_qty = bid_qty + ask_qty

        # Determine probabilities for each action
        p_buy = np.clip(self.posterior_mean / mid_price, 0, 1) if mid_price != 0 else 0
        p_sell = 1 - p_buy

        # Adjust probabilities based on BBP, decay factor, posterior std, and order quantity
        if bbp < 0.5 and self.posterior_std < self.std_valuation and current_qty > self.mean_qty:
            p_buy *= 1.1  # increase buy probability
        elif bbp > 0.5 and self.posterior_std > self.std_valuation and decay_factor > 0.5 and current_qty < self.mean_qty:
            p_sell *= 1.1  # increase sell probability

        # Normalize the probabilities so they sum to 1
        total_prob = p_buy + p_sell
        p_buy /= total_prob
        p_sell /= total_prob

        decision = np.random.choice(['BUY', 'SELL'], p=[p_buy, p_sell])

        return decision
